# Introdução

Essa análise eu tive que desenvolver para um processo seletivo genew da Accenture em 2024.
Apesar de eu não ter sido selecionado, eu achei esse case bem interessante para desenvolver minhas habilidades com as ferramentas Pandas, assim como, tomar decisões com base em dados de um setor essencial em quase todas as industrias, que é o setor de Supply Chain.

O enunciado dos problemas estão disponiveis na apresentação final em powerpoint (que infelizmente, não tive a chance de apresentar para os recrutadores);
]
O dataset que foi utilizado está disponivel no link: https://drive.google.com/drive/folders/1GjBeS_8IiRR13euMGChP6nDPDI9ezjSw?usp=sharing

In [1]:
import math
import pandas as pd
import numpy as np
import re

# Problema 1

## Questão 1

In [2]:
sales23 = pd.read_csv('genew1.csv') #DataFrame das vendas reais em 2023

forecast = pd.read_csv("Forecast.csv") #DataFrame do Forecast

df = sales23.copy()


In [3]:
#Organizando e limpando a vizualização dos nossos dados
lista = []

for i in range(len(df)):
  temp = df['Mês'].iloc[i][0:3]
  lista.append(temp)

newdata = pd.Series(lista)
newdata = newdata.str.lower()
data = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro']

df['Mês'] = newdata
df['Mês'] = df['Mês'].replace(["jan","fev","mar","abr",'mai','jun','jul','ago','set','out','nov','dez'],data)


df = df.groupby(['SKU','Mês'])['Quantidade Vendida (unidades)'].sum().reset_index()
df['Mês'] = pd.Categorical(df['Mês'], categories=data, ordered=True)

df = df.sort_values(['SKU','Mês']).reset_index()
del df['index']

df['FORECAST'] = forecast['FORECAST (unidades)']
df['ERRO'] = (df['Quantidade Vendida (unidades)'] - df['FORECAST'])
#df['|A - F / A|'] = np.abs((df['ERRO'] / df['Quantidade Vendida (unidades)']))
df['|A - F / A|%'] = np.abs((df['ERRO'] / df['Quantidade Vendida (unidades)'])*100)

df.to_excel('Q1_1.xlsx', index = False)
df #Tabela 1 - Forecast 2023

,SKU,Mês,Quantidade Vendida (unidades),FORECAST,ERRO,|A - F / A|%
0,12403,Janeiro,617.0,1100,-483.0,78.282010
1,12403,Fevereiro,1018.0,1612,-594.0,58.349705
2,12403,Março,477.0,1108,-631.0,132.285115
3,12403,Abril,597.0,1070,-473.0,79.229481
4,12403,Maio,2027.0,2582,-555.0,27.380365
5,12403,Junho,1367.0,2067,-700.0,51.207023
6,12403,Julho,1902.0,2392,-490.0,25.762355
7,12403,Agosto,1088.0,1598,-510.0,46.875000
8,12403,Setembro,798.0,1430,-632.0,79.197995
9,12403,Outubro,938.0,1430,-492.0,52.452026


In [4]:
#Fazendo o desvio padrão do erro de cada produto
prod1 = df.copy()
mask = prod1["SKU"] == 12403
prod1 = df.where(mask).dropna()

prod1 = prod1.reset_index()
del prod1['index']
std2023_1 = np.std(prod1['ERRO']) #Desvio Padrão 1


prod2 = df.copy()
mask = prod2["SKU"] == 12404
prod2 = df.where(mask).dropna()

prod2 = prod2.reset_index()
del prod2['index']
std2023_2 = np.std(prod2['ERRO']) #Desvio padrão 2

prod3 = df.copy()
mask = prod3["SKU"] == 12405
prod3 = df.where(mask).dropna()

prod3 = prod3.reset_index()
del prod3['index']
std2023_3 = np.std(prod3['ERRO']) #Desvio Padrão 3

std = [std2023_1, std2023_2, std2023_3]

In [5]:
#Calculando e montando a tabela MAPE
mape = df.groupby('SKU')['|A - F / A|%'].sum()/len(df)
mape.reset_index()
mape = pd.DataFrame(mape)
mape.columns = ['MAPE']
mape['Desvio Padrão'] = std

mape.to_excel('mape1.xlsx', index = False)
mape

,MAPE,Desvio Padrão
SKU,,
12403,21.574261,81.105727
12404,11.263686,67.292892
12405,10.841292,73.371013


## Questão 2

In [6]:
#Função para fazer o forecast 2024 através dos valores de 2023
def mediaMovel(dataframe, intervalo, column = str):
  df = dataframe.copy()
  df = df.reset_index()
  del df['index']
  del df['FORECAST']
  del df['ERRO']
  del df['|A - F / A|%']
  pointer = 0

  for i in range(intervalo + 1):
    pointer = intervalo - i
    if pointer > 0:
      temp1 = df[-pointer:]
      temp2 = df[0:i]
    else:
      temp1 = df[-pointer:0]
      temp2 = df[0:i]

    aux = pd.concat([temp1,temp2]).reset_index()
    media = np.mean(aux[column])
    df.loc[i, column] = media

  for i in range(intervalo + 1, len(df)):
    temp1 = df[i - intervalo:i]
    media = np.mean(temp1[column])
    df.loc[i,column] = media

  return df


In [7]:
#Calculando as médias móveis, erros e seus respectivos desvios padrões por produto
lista = []

#média móvel das vendas do produto 1:
sku03 = mediaMovel(prod1, 3,'Quantidade Vendida (unidades)')
sku03['ERRO'] = prod1['Quantidade Vendida (unidades)'] - sku03['Quantidade Vendida (unidades)']
std1 = np.std(sku03['ERRO'])
lista.append(sku03)

#media movel do produto 2:
sku04 = mediaMovel(prod2, 3, 'Quantidade Vendida (unidades)')
sku04['ERRO'] = (prod2['Quantidade Vendida (unidades)'] - sku04['Quantidade Vendida (unidades)'])
std2 = np.std(sku04['ERRO'])
lista.append(sku04)

#media movel do produto 3:
sku05 = mediaMovel(prod3, 3, 'Quantidade Vendida (unidades)')
sku05['ERRO'] = (prod3['Quantidade Vendida (unidades)'] - sku05['Quantidade Vendida (unidades)'])
std3 = np.std(sku05['ERRO'])
lista.append(sku05)

std = [std1, std2, std3]

forecast2024 = pd.concat(lista, ignore_index = True)
forecast2024 = forecast2024.rename(columns={'Quantidade Vendida (unidades)': 'FORECAST 2024'})

#Realizando a MAPE para esse tipo de análise com a média móvel em relação ao ano de 2023
forecast2024['|A - F / A|%'] = np.abs((forecast2024['ERRO'] / df['Quantidade Vendida (unidades)'])*100)

#para cada produto

#Calculando e montando a tabela MAPE
mape = forecast2024.groupby('SKU')['|A - F / A|%'].sum()/len(df)
mape.reset_index()
mape = pd.DataFrame(mape)
mape.columns = ['MAPE']
mape['Desvio Padrão'] = std

forecast2024['FORECAST 2024'] = forecast2024['FORECAST 2024'].astype(int)

forecast2024.to_excel('forecast2024.xlsx', index = False)
forecast2024


#Tabela 2 - Forescast 2024


,SKU,Mês,FORECAST 2024,ERRO,|A - F / A|%
0,12403.0,Janeiro,750,-133.000000,21.555916
1,12403.0,Fevereiro,687,330.666667,32.481991
2,12403.0,Março,660,-183.111111,38.388074
3,12403.0,Abril,699,-102.148148,17.110243
4,12403.0,Maio,682,1344.802469,66.344473
5,12403.0,Junho,680,686.514403,50.220512
6,12403.0,Julho,687,1214.722908,63.865558
7,12403.0,Agosto,683,404.679927,37.194846
8,12403.0,Setembro,683,114.305746,14.324028
9,12403.0,Outubro,684,253.236194,26.997462


In [8]:
mape

,MAPE,Desvio Padrão
SKU,,
12403.0,11.264902,495.261068
12404.0,69.835782,882.639274
12405.0,8.088565,188.335826


# Problema 2

## Questão 1

In [10]:
suppliers = pd.read_csv('fornecedores.csv')
invoice = pd.read_csv('SupplyFaturamento.csv')
otif = pd.read_csv('supplyOTIF.csv')
bom = pd.read_csv('bom.csv')

In [11]:
df = suppliers.copy()
pd.options.display.float_format = '{:.2f}'.format #Somente para evitar a notação cientifica

df = pd.merge(df, invoice, how='outer', on = 'Fornecedor')
df = pd.merge(df, otif, how='outer', on = 'Fornecedor')
df = df.rename(columns={'Valor de Venda unitário (R$)\ncom impostos': 'Valor Unitário (R$)',
                        'Capacidade Entrega': 'Capacidade Entrega (Ano)',
                        'Item' : 'Componente',
                        'Pedido Mínimo\n(unidades)' : 'Pedido Mínimo (Und)',
                        'OTIF 2023' : 'OTIF 2023(%)',
                        'Leadtime de Entrega (dias)' : 'Leadtime (dias)'
                        })
df = df.replace(['N.A','-'], np.nan) #Trocando os Nan


df['OTIF 2023(%)'] = df['OTIF 2023(%)'].str.replace('%','').astype(float)
df['Capacidade Entrega (Ano)'] = df['Capacidade Entrega (Ano)'].str.replace('/ano','').str.replace('.','').astype(float)
df['Valor Unitário (R$)'] = df['Valor Unitário (R$)'].str.replace('$ ', '').str.replace('.','').str.replace(',','.').astype(float)
df['Faturamento 2023 (R$)'] = df['Faturamento 2023 (R$)'].str.replace('$ ', '').str.replace('.','').str.replace(',','.').astype(float)
df['Frete (R$/und)'] = df['Frete (R$/und)'].str.replace('$ ', '').str.replace(',','.').astype(float)
df

,Fornecedor,Localização,Componente,Capacidade Entrega (Ano),Pedido Mínimo (Und),Valor Unitário (R$),Modalidade Entrega,Modal,Frete (R$/und),Leadtime (dias),Faturamento 2023 (R$),OTIF 2023(%)
0,Techlink Electronics,Taiwan,Processador A1,100000.00,100.00,1390.00,CIF,Maritimo,NaN,80,2000000000.00,NaN
1,Techlink Electronics,Taiwan,NaN,NaN,NaN,1750.00,CIF,Aereo,NaN,20,2000000000.00,NaN
2,Techlink Electronics,Taiwan,Periferico Entrada+Saida,1000000.00,500.00,5.50,FOB,Marítimo,5.00,130,2000000000.00,NaN
3,Techlink Electronics,Taiwan,Temporizador XJ01,1000000.00,200.00,58.00,FOB,Aereo,15.00,30,2000000000.00,NaN
4,Techlink Electronics,Taiwan,Processador Gráfico B33,1000000.00,200.00,110.00,FOB,Aereo,30.00,30,2000000000.00,NaN
5,Techlink Electronics,Taiwan,Memória Vídeo 128k,1000000.00,200.00,340.00,FOB,Aereo,80.00,30,2000000000.00,NaN
6,Techlink Electronics,Taiwan,Termistor PP,1000000.00,200.00,1.75,FOB,Maritimo,3.30,130,2000000000.00,NaN
7,Techlink Electronics,Taiwan,Cabo Sensor Termopar 0.2m,1000000.00,200.00,11.00,FOB,Maritimo,5.00,130,2000000000.00,NaN
8,Nexus Tech Manufacturing,Argentina,Processador A1,75000.00,20.00,1800.00,CIF,NaN,NaN,30,750000000.00,83.00
9,Nexus Tech Manufacturing,Argentina,Memória Vídeo 128k,40000.00,20.00,399.00,FOB,Rodoviário,15.00,50,750000000.00,83.00


In [12]:
bom = bom.rename(columns={'Quantidade Necessária \n(Para 1 und produto final)': 'Qtde para 1 und'})
bom['Qtde para 1 und'] = bom['Qtde para 1 und'].astype(int)

sku03['Quantidade Vendida (unidades)'] = sku03['Quantidade Vendida (unidades)'].astype(int)
total03 = sku03.groupby('SKU')['Quantidade Vendida (unidades)'].sum()

sku04['Quantidade Vendida (unidades)'] = sku04['Quantidade Vendida (unidades)'].astype(int)
total04 = sku04.groupby('SKU')['Quantidade Vendida (unidades)'].sum()

sku05['Quantidade Vendida (unidades)'] = sku05['Quantidade Vendida (unidades)'].astype(int)
total05 = sku05.groupby('SKU')['Quantidade Vendida (unidades)'].sum()

total2024 = [int(total03.iloc[0]), int(total04.iloc[0]), int(total05.iloc[0])]

mask1 = bom['Produto Final'] == 'Microcontrolador XJ23'
mask2 = bom['Produto Final'] == 'Placa YY3'
mask3 = bom['Produto Final'] == 'Sensor de Temperatura PP2'

aux1 = bom.where(mask1).dropna()
aux2 = bom.where(mask2).dropna()
aux3 = bom.where(mask3).dropna()

#Multiplicando o total pelas unidades necessárias
aux1['Qtde para 1 und'] = aux1['Qtde para 1 und'] * total2024[0]
aux2['Qtde para 1 und'] = aux2['Qtde para 1 und'] * total2024[1]
aux3['Qtde para 1 und'] = aux3['Qtde para 1 und'] * total2024[2]

aux = [aux1, aux2, aux3]

bom['DEMANDA 2024'] = pd.concat(aux, ignore_index= True)['Qtde para 1 und']
bom


,Produto Final,Componente,Qtde para 1 und,DEMANDA 2024
0,Microcontrolador XJ23,Processador A1,1,8262.00
1,Microcontrolador XJ23,Periferico Entrada+Saida,2,16524.00
2,Microcontrolador XJ23,Temporizador XJ01,2,16524.00
3,Placa YY3,Processador Gráfico B33,1,32625.00
4,Placa YY3,Memória Vídeo 128k,1,32625.00
5,Sensor de Temperatura PP2,Termistor PP,1,8689.00
6,Sensor de Temperatura PP2,Cabo Sensor Termopar 0.2m,2,17378.00


## Questão 2

In [13]:
cost = df.copy()
cost = cost[['Fornecedor','Componente','Pedido Mínimo (Und)','Valor Unitário (R$)','Frete (R$/und)','Faturamento 2023 (R$)','OTIF 2023(%)','Leadtime (dias)']]
cost = pd.merge(cost,bom, on= 'Componente', how='inner')



In [14]:
#Agora verificando os candidatos aptos para ser nossos fornecedores
cost['TOTAL (R$)'] = cost['Valor Unitário (R$)'].astype(float) * cost['DEMANDA 2024']

cost = cost[cost['TOTAL (R$)'] < cost['Faturamento 2023 (R$)'] * 3/10]
cost = cost[cost['Leadtime (dias)'] <= 50]

cost = cost.reset_index()
del cost['index']

#Removendo o fornecedor que não atende a OTIF
cost = cost.drop(index = 8)
cost = cost.drop(index = 10)
cost = cost.reset_index()
del cost['index']
cost

#Tabela ? - Candidatos a fornecedor

,Fornecedor,Componente,Pedido Mínimo (Und),Valor Unitário (R$),Frete (R$/und),Faturamento 2023 (R$),OTIF 2023(%),Leadtime (dias),Produto Final,Qtde para 1 und,DEMANDA 2024,TOTAL (R$)
0,Nexus Tech Manufacturing,Processador A1,20.00,1800.00,NaN,750000000.00,83.00,30,Microcontrolador XJ23,1,8262.00,14871600.00
1,InnovateX Supplies,Periferico Entrada+Saida,10.00,8.90,9.90,3000000.00,NaN,20,Microcontrolador XJ23,2,16524.00,147063.60
2,Techlink Electronics,Temporizador XJ01,200.00,58.00,15.00,2000000000.00,NaN,30,Microcontrolador XJ23,2,16524.00,958392.00
3,Techlink Electronics,Processador Gráfico B33,200.00,110.00,30.00,2000000000.00,NaN,30,Placa YY3,1,32625.00,3588750.00
4,Techlink Electronics,Memória Vídeo 128k,200.00,340.00,80.00,2000000000.00,NaN,30,Placa YY3,1,32625.00,11092500.00
5,Nexus Tech Manufacturing,Memória Vídeo 128k,20.00,399.00,15.00,750000000.00,83.00,50,Placa YY3,1,32625.00,13017375.00
6,Nexus Tech Manufacturing,Termistor PP,20.00,3.80,12.00,750000000.00,83.00,50,Sensor de Temperatura PP2,1,8689.00,33018.20
7,InnovateX Supplies,Termistor PP,10.00,4.50,9.90,3000000.00,NaN,20,Sensor de Temperatura PP2,1,8689.00,39100.50
8,InnovateX Supplies,Cabo Sensor Termopar 0.2m,10.00,13.00,9.90,3000000.00,NaN,20,Sensor de Temperatura PP2,2,17378.00,225914.00


In [15]:
#Encontrando o fornecedor mais barato
temp = cost.groupby('Componente')[['Fornecedor','Frete (R$/und)','Leadtime (dias)','DEMANDA 2024','Valor Unitário (R$)','TOTAL (R$)']].min().reset_index()
temp

#Tabela ? -

,Componente,Fornecedor,Frete (R$/und),Leadtime (dias),DEMANDA 2024,Valor Unitário (R$),TOTAL (R$)
0,Cabo Sensor Termopar 0.2m,InnovateX Supplies,9.90,20,17378.00,13.00,225914.00
1,Memória Vídeo 128k,Nexus Tech Manufacturing,15.00,30,32625.00,340.00,11092500.00
2,Periferico Entrada+Saida,InnovateX Supplies,9.90,20,16524.00,8.90,147063.60
3,Processador A1,Nexus Tech Manufacturing,NaN,30,8262.00,1800.00,14871600.00
4,Processador Gráfico B33,Techlink Electronics,30.00,30,32625.00,110.00,3588750.00
5,Temporizador XJ01,Techlink Electronics,15.00,30,16524.00,58.00,958392.00
6,Termistor PP,InnovateX Supplies,9.90,20,8689.00,3.80,33018.20


In [16]:
temp = temp.drop(columns= ['DEMANDA 2024'])

In [17]:
#Agora a tabela Bom com os fornecedores definidos até o momento (Ainda não foi avaliado o frete)
bom = pd.merge(bom,temp, on= 'Componente', how='inner')
bom

,Produto Final,Componente,Qtde para 1 und,DEMANDA 2024,Fornecedor,Frete (R$/und),Leadtime (dias),Valor Unitário (R$),TOTAL (R$)
0,Microcontrolador XJ23,Processador A1,1,8262.00,Nexus Tech Manufacturing,NaN,30,1800.00,14871600.00
1,Microcontrolador XJ23,Periferico Entrada+Saida,2,16524.00,InnovateX Supplies,9.90,20,8.90,147063.60
2,Microcontrolador XJ23,Temporizador XJ01,2,16524.00,Techlink Electronics,15.00,30,58.00,958392.00
3,Placa YY3,Processador Gráfico B33,1,32625.00,Techlink Electronics,30.00,30,110.00,3588750.00
4,Placa YY3,Memória Vídeo 128k,1,32625.00,Nexus Tech Manufacturing,15.00,30,340.00,11092500.00
5,Sensor de Temperatura PP2,Termistor PP,1,8689.00,InnovateX Supplies,9.90,20,3.80,33018.20
6,Sensor de Temperatura PP2,Cabo Sensor Termopar 0.2m,2,17378.00,InnovateX Supplies,9.90,20,13.00,225914.00


## Questão 3

In [18]:
bom = bom.replace(np.nan, 0)
bom['Total + Frete (R$)'] = bom['TOTAL (R$)'] + bom['Frete (R$/und)'].astype(float) * bom['DEMANDA 2024']
total = bom['Total + Frete (R$)'].sum()
bom

,Produto Final,Componente,Qtde para 1 und,DEMANDA 2024,Fornecedor,Frete (R$/und),Leadtime (dias),Valor Unitário (R$),TOTAL (R$),Total + Frete (R$)
0,Microcontrolador XJ23,Processador A1,1,8262.00,Nexus Tech Manufacturing,0.00,30,1800.00,14871600.00,14871600.00
1,Microcontrolador XJ23,Periferico Entrada+Saida,2,16524.00,InnovateX Supplies,9.90,20,8.90,147063.60,310651.20
2,Microcontrolador XJ23,Temporizador XJ01,2,16524.00,Techlink Electronics,15.00,30,58.00,958392.00,1206252.00
3,Placa YY3,Processador Gráfico B33,1,32625.00,Techlink Electronics,30.00,30,110.00,3588750.00,4567500.00
4,Placa YY3,Memória Vídeo 128k,1,32625.00,Nexus Tech Manufacturing,15.00,30,340.00,11092500.00,11581875.00
5,Sensor de Temperatura PP2,Termistor PP,1,8689.00,InnovateX Supplies,9.90,20,3.80,33018.20,119039.30
6,Sensor de Temperatura PP2,Cabo Sensor Termopar 0.2m,2,17378.00,InnovateX Supplies,9.90,20,13.00,225914.00,397956.20


In [23]:
print('Total + Frete para 2024 (R$): ',total)

Total + Frete para 2024 (R$):  33054873.7


# Problema 3


## Questão 1


In [20]:
#O estoque de ciclo é dado pela os insumos necessários durante o leadtime de ressuprimento
stock = bom.copy()
stock = stock[['Produto Final','Fornecedor','Componente', 'Qtde para 1 und', 'DEMANDA 2024','Leadtime (dias)']]
stock['Estoque Ciclo'] = ((stock['DEMANDA 2024']/365)*stock['Leadtime (dias)'])/2
stock['Estoque Ciclo'] = stock['Estoque Ciclo'].astype(int)
#Estoque de segurança
z = 1.6448

stock['Estoque Segurança'] = 0


filtro = stock['Produto Final'] == 'Microcontrolador XJ23'
indices = stock.index[filtro].tolist()
#Vou multiplicar as colunas manualmente
for i in indices:
  raizLeadtime = stock.loc[i, 'Leadtime (dias)']
  stock.loc[i , 'Estoque Segurança'] =  z * std2023_1 * math.sqrt(raizLeadtime)


filtro = stock['Produto Final'] == 'Placa YY3'
indices = stock.index[filtro].tolist()

for i in indices:
  raizLeadtime = stock.loc[i, 'Leadtime (dias)']
  stock.loc[i , 'Estoque Segurança'] =  z * std2023_2 * math.sqrt(raizLeadtime)


filtro = stock['Produto Final'] == 'Sensor de Temperatura PP2'
indices = stock.index[filtro].tolist()

for i in indices:
  raizLeadtime = stock.loc[i, 'Leadtime (dias)']
  stock.loc[i , 'Estoque Segurança'] =  z * std2023_3 * math.sqrt(raizLeadtime)

stock['Estoque Segurança'] = stock['Estoque Segurança'].astype(int)

stock['Estoque Total'] = stock['Estoque Segurança'] + stock['Estoque Ciclo']
stock

,Produto Final,Fornecedor,Componente,Qtde para 1 und,DEMANDA 2024,Leadtime (dias),Estoque Ciclo,Estoque Segurança,Estoque Total
0,Microcontrolador XJ23,Nexus Tech Manufacturing,Processador A1,1,8262.00,30,339,730,1069
1,Microcontrolador XJ23,InnovateX Supplies,Periferico Entrada+Saida,2,16524.00,20,452,596,1048
2,Microcontrolador XJ23,Techlink Electronics,Temporizador XJ01,2,16524.00,30,679,730,1409
3,Placa YY3,Techlink Electronics,Processador Gráfico B33,1,32625.00,30,1340,606,1946
4,Placa YY3,Nexus Tech Manufacturing,Memória Vídeo 128k,1,32625.00,30,1340,606,1946
5,Sensor de Temperatura PP2,InnovateX Supplies,Termistor PP,1,8689.00,20,238,539,777
6,Sensor de Temperatura PP2,InnovateX Supplies,Cabo Sensor Termopar 0.2m,2,17378.00,20,476,539,1015


## Questão 2

In [21]:
df = pd.read_csv('Estoque2023.csv')
df = df.rename(columns={'Descrição' : 'Componente',
                        'Quantidade em Estoque (und)' : 'Qtnd Estoque 2023'})

df = df[['Componente','Qtnd Estoque 2023']]
df = pd.merge(df,stock, on= 'Componente', how='inner')

df = df[['Componente','Qtnd Estoque 2023','Estoque Total']]
df = df.rename(columns={'Estoque Total' : 'Estoque 2024'})

inventoryCost = 32

df['Custo 2023 (R$)'] = df['Qtnd Estoque 2023'] * inventoryCost
df['Custo 2024 (R$)'] = df['Estoque 2024'] * inventoryCost
df['Diferença'] = df['Custo 2023 (R$)'] - df['Custo 2024 (R$)']
econonomiaTotal = df['Diferença'].sum()
df

,Componente,Qtnd Estoque 2023,Estoque 2024,Custo 2023 (R$),Custo 2024 (R$),Diferença
0,Periferico Entrada+Saida,1117,1048,35744,33536,2208
1,Temporizador XJ01,1543,1409,49376,45088,4288
2,Processador Gráfico B33,2830,1946,90560,62272,28288
3,Memória Vídeo 128k,3103,1946,99296,62272,37024
4,Termistor PP,1750,777,56000,24864,31136
5,Cabo Sensor Termopar 0.2m,2750,1015,88000,32480,55520
6,Processador A1,1691,1069,54112,34208,19904


In [22]:
print("Economia Total (R$): ", econonomiaTotal)

Economia Total (R$):  178368
